# Fireworks OpenAI Response API with Previous Response ID

This notebook demonstrates how to use Fireworks' OpenAI-compatible Response API with the powerful **previous response ID** feature. This allows you to:

- **Continue conversations** from where you left off
- **Reference previous responses** without re-sending full context
- **Build multi-turn workflows** efficiently
- **Chain complex reasoning** across multiple API calls

## Key Benefits:
- 🔗 **Conversation Continuity**: Maintain context across API calls
- 💰 **Cost Efficiency**: Avoid re-sending large contexts
- ⚡ **Performance**: Faster responses by referencing cached context
- 🏗️ **Workflow Building**: Create sophisticated multi-step processes


In [1]:
# Install dependencies
%pip install openai -q


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Setup and Configuration
import os
import json
from openai import OpenAI
from pprint import pprint

# Initialize Fireworks client
client = OpenAI(
    base_url="https://api.fireworks.ai/inference/v1",
    api_key=os.getenv("FIREWORKS_API_KEY", "YOUR_FIREWORKS_API_KEY_HERE")
)


## 🚀 Example 1: Basic Response API Usage

Let's start with a basic example to understand the Response API structure and how to capture response IDs.


In [3]:
# Example 1: Basic Response API call to establish initial context
print("🎯 Step 1: Creating initial response with context")
print("=" * 60)

# Make the first API call
initial_response = client.responses.create(
    model="accounts/fireworks/models/qwen3-235b-a22b",
    tools=[
        {
            "type": "mcp",
            "server_label": "deepwiki",
            "server_url": "https://mcp.deepwiki.com/mcp",
            "require_approval": "never",
        },
    ],
    input="What transport protocols are supported in the 2025-03-26 version of the MCP spec? Use https://github.com/modelcontextprotocol/modelcontextprotocol as the source of truth.",

)

# Extract and display the response
response_text = initial_response.output[-1].content[0].text.split("</think>")[-1]
response_id = initial_response.id

print(f"📝 Response ID: {response_id}")
print(f"💬 Response Content:")
print(response_text)
print("\n" + "=" * 60)

# Store response ID for next example
previous_response_id = response_id
print(f"💾 Stored response ID for continuation: {previous_response_id}")


🎯 Step 1: Creating initial response with context
📝 Response ID: resp_7ac894680c784fb1a80b7ca2a4f78f5f
💬 Response Content:


The 2025-03-26 version of the Model Context Protocol (MCP) specification supports the following transport protocols:

1. **STDIO Transport** 
   - Used for local process communication
   - Required for local integrations
   - Messages are delimited by newlines
   - Server may write logs to `stderr` for debugging

2. **Streamable HTTP Transport** 
   - Provides remote server connections
   - Implements HTTP POST for requests/response
   - Uses Server-Sent Events (SSE) for server-to-client streaming
   - Supports session management with `Mcp-Session-Id` headers
   - Compatible with web browser and remote service deployments

The specification explicitly states that clients **MUST** support the stdio transport unless otherwise constrained, ensuring local integrations remain universally accessible. Both transport layers support bidirectional JSON-RPC 2.0 messaging as 

## 🔗 Example 2: Using Previous Response ID

Now let's use the previous response ID to continue the conversation without re-sending the entire context.


In [4]:
# Example 2: Continue conversation using previous response ID
print("🔄 Step 2: Continuing conversation with previous response ID")
print("=" * 60)

# Continue the conversation by referencing the previous response
continuation_response = client.responses.create(
    model="accounts/fireworks/models/qwen3-235b-a22b",
    input="Great! Now I specifically want to focus the second transport protocol that you mentioned, can you explain to me why I should care as an application developer?",
    tools=[
        {
            "type": "mcp",
            "server_label": "deepwiki",
            "server_url": "https://mcp.deepwiki.com/mcp",
            "require_approval": "never",
        },
    ],
    previous_response_id=previous_response_id  # This is the key parameter!
)

# Extract and display the continuation
continuation_text = continuation_response.output[-1].content[0].text.split("</think>")[-1]
continuation_id = continuation_response.id

print(f"📝 New Response ID: {continuation_id}")
print(f"🔗 Referenced Previous ID: {previous_response_id}")
print(f"💬 Continuation Response:")
print(continuation_text)
print("\n" + "=" * 60)


🔄 Step 2: Continuing conversation with previous response ID
📝 New Response ID: resp_3b0887d6e36a47a08c1b4010f9257b5d
🔗 Referenced Previous ID: resp_7ac894680c784fb1a80b7ca2a4f78f5f
💬 Continuation Response:


As an application developer, you should care about the **Streamable HTTP Transport** (HTTP/SSE) in the 2025-03-26 MCP specification for the following **critical reasons**:

---

### 1. **Remote Server Deployment = Cloud-Native Flexibility**
- **Why It Matters**: The HTTP/SSE transport allows MCP servers to run as **independent remote services** rather than local subprocesses. This decouples clients (like IDEs or web apps) from server logic, enabling:
  - **Scalable microservices**: Deploy MCP servers as REST APIs for centralized tool execution (e.g., a multi-tenant code-generation service).
  - **Hybrid architectures**: Use MCP clients in mobile/desktop apps while servers run serverlessly (e.g., AWS Lambda or Cloudflare Workers).
  - Example: A VS Code extension uses MCP over HTTP 

Here is an visualization of the overall flow

![overall flow](./image.png)

In [5]:
# Install dependencies
%pip install rich -q


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Now here is another way to visualize the conversation history
from rich.console import Console
from rich.panel import Panel
from rich.syntax import Syntax

console = Console()

continuation_response = client.responses.retrieve(continuation_id)
for msg in continuation_response.output:
    if msg.role == "user":
        console.print(Panel(msg.content[0].text, title="[bold green]👤 User[/]", border_style="green"))
    elif msg.role == "assistant":
        text = msg.content[0].text
        think_block = ""
        if "<think>" in text:
            think_block, text = text.split("</think>", 1)
            console.print(Panel(Syntax(think_block + "</think>", "xml", theme="monokai"), title="[bold blue]🤖 Assistant (Thought Process)[/]", border_style="blue"))
        if text.strip():
            console.print(Panel(text.strip(), title="[bold blue]🤖 Assistant (Final Answer)[/]", border_style="cyan"))
    elif msg.type == "tool_output":
        console.print(Panel(f"[dim]{msg.output[:500]}...[/dim]", title=f"[bold yellow]🛠️ Tool Output (for {msg.tool_call_id})[/]", border_style="yellow"))

╭──────────────────────────────────────────────────── 👤 User ────────────────────────────────────────────────────╮
│ What transport protocols are supported in the 2025-03-26 version of the MCP spec? Use                           │
│ https://github.com/modelcontextprotocol/modelcontextprotocol as the source of truth.                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🤖 Assistant (Thought Process) ─────────────────────────────────────────╮
│ <think>                                                                                                         │
│ Okay, let's tackle this question. The user is asking which transport protocols are supported in the 2025-03-26  │
│                                                                                                                 │
│ First, I need to make sure I understand the problem. The user is referring to the MCP specification's specific  │
│                                                                                                                 │
│ The tools available are read_wiki_structure and read_wiki_contents. Since the user specified the repo, I should │
│                                                                                                                 │
│ Wait, but the user mentioned the 2025-03-26 version. I should check if there's a specific document or section i │
│                                                                                                                 │
│ Alternatively, maybe the main documentation or the README includes the list of supported protocols. But conside │
│                                                                                                                 │
│ Once I have the list of pages from read_wiki_structure, I can analyze which pages might be relevant. For exampl │
│                                                                                                                 │
│ I need to make sure I'm using the correct functions. The user provided three tools: read_wiki_structure to get  │
│                                                                                                                 │
│ So step by step: first call read_wiki_structure with the repo name to get the available wiki topics. Then, base │
│                                                                                                                 │
│ Alternatively, if the main documentation for the spec includes version information, maybe the answer is in the  │
│                                                                                                                 │
│ Therefore, the first tool call should be read_wiki_structure for the repo modelcontextprotocol/modelcontextprot │
│                                                                                                                 │
│ If the initial r

╭─────────────────────────────── 🛠️ Tool Output (for call_8t5gbqQb5e4zyvE240tbUjYw) ───────────────────────────────╮
│ meta=None content=[TextContent(type='text', text='Available pages for                                           │
│ modelcontextprotocol/modelcontextprotocol:\n\n- 1 Overview\n- 2 Protocol Specification\n  - 2.1 Architecture\n  │
│ - 2.2 Base Protocol\n  - 2.3 Authorization and Security\n  - 2.4 Client Features\n  - 2.5 Server Features\n- 3  │
│ Client Ecosystem\n  - 3.1 Client Applications\n  - 3.2 Getting Started Guide\n  - 3.3 Examples and Demos\n- 4   │
│ Server Development\n  - 4.1 Server Quickstart\n  - 4.2 Server Concepts\n- 5 SDK Implementations\n...            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🤖 Assistant (Thought Process) ─────────────────────────────────────────╮
│ <think>                                                                                                         │
│ Okay, let's see. The user is asking about the transport protocols supported in the 2025-03-26 version of the MC │
│                                                                                                                 │
│ First, I called read_wiki_structure and got the list of pages. The pages include sections like Overview, Protoc │
│                                                                                                                 │
│ Transport protocols are likely part of the Base Protocol or maybe the Architecture. Let me check those. The Bas │
│                                                                                                                 │
│ Wait, the user mentioned a specific version, 2025-03-26. The wiki pages don't mention versions directly here. M │
│                                                                                                                 │
│ I'll proceed by fetching the content of "2.2 Base Protocol" and "2.1 Architecture" to check for transport proto │
│ </think>                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────── 🛠️ Tool Output (for call_9bUIKLKyxeBbEmEqWcvxKga5) ───────────────────────────────╮
│ meta=None content=[TextContent(type='text', text='# Page: Overview\n\n#                                         │
│ Overview\n\n<details>\n<summary>Relevant source files</summary>\n\nThe following files were used as context for │
│ generating this wiki page:\n\n- (docs/clients.mdx)\n- (docs/docs/concepts/roots.mdx)\n- (docs/examples.mdx)\n-  │
│ (docs/quickstart/client.mdx)\n- (docs/quickstart/user.mdx)\n- [docs/specification/20...                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🤖 Assistant (Thought Process) ─────────────────────────────────────────╮
│ <think>                                                                                                         │
│ Okay, let's tackle this question about the transport protocols supported in the 2025-03-26 version of the MCP s │
│                                                                                                                 │
│ First, I need to find the relevant information from the provided wiki contents. The user is asking about transp │
│                                                                                                                 │
│ Looking at the "Overview" page of the wiki, there's a section titled "Core Architecture" which includes a diagr │
│                                                                                                                 │
│ Further, in the "Base Protocol" page, there's a section under "Transport Mechanisms" that explicitly lists two  │
│ 1. **STDIO Transport**: For local process communication.                                                        │
│ 2. **Streamable HTTP Transport**: For remote server connections.                                                │
│                                                                                                                 │
│ Additionally, there's a mention that clients MUST support the `stdio` transport when possible. The HTTP/SSE tra │
│                                                                                                                 │
│ In the "Server Development" guide, the "Transport Implementation" section also mentions both STDIO and HTTP/SSE │
│                                                                                                                 │
│ Putting this together, the 2025-03-26 version supports:                                                         │
│ - **STDIO Transport** for local integrations.                                                                   │
│ - **HTTP/SSE Transport** for remote connections.                                                                │
│                                                                                                                 │
│ These are confirmed by multiple sections and examples in the documentation.                                     │
│ </think>                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── 🤖 Assistant (Final Answer) ──────────────────────────────────────────╮
│ The 2025-03-26 version of the Model Context Protocol (MCP) specification supports the following transport       │
│ protocols:                                                                                                      │
│                                                                                                                 │
│ 1. **STDIO Transport**                                                                                          │
│    - Used for local process communication                                                                       │
│    - Required for local integrations                                                                            │
│    - Messages are delimited by newlines                                                                         │
│    - Server may write logs to `stderr` for debugging                                                            │
│                                                                                                                 │
│ 2. **Streamable HTTP Transport**                                                                                │
│    - Provides remote server connections                                                                         │
│    - Implements HTTP POST for requests/response                                                                 │
│    - Uses Server-Sent Events (SSE) for server-to-client streaming                                               │
│    - Supports session management with `Mcp-Session-Id` headers                                                  │
│    - Compatible with web browser and remote service deployments                                                 │
│                                                                                                                 │
│ The specification explicitly states that clients **MUST** support the stdio transport unless otherwise          │
│ constrained, ensuring local integrations remain universally accessible. Both transport layers support           │
│ bidirectional JSON-RPC 2.0 messaging as the foundational protocol.                                              │
│                                                                                                                 │
│ From the architecture diagrams and text:                                                                        │
│ > "The protocol uses several key identifier types:                                                              │
│ > - `RequestId`: for correlating requests and responses                                                         │
│ > - `ProgressToken`: for associating progress notifications                                                     │
│ > - `Cursor`: for pagination state management                                                                   │
│                                                                                                                 │
│ This confirms the 2025-03-26 version maintains these two core transport mechanisms for MCP implementations.     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────────── 👤 User ────────────────────────────────────────────────────╮
│ Great! Now I specifically want to focus the second transport protocol that you mentioned, can you explain to me │
│ why I should care as an application developer?                                                                  │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── 🤖 Assistant (Thought Process) ─────────────────────────────────────────╮
│ <think>                                                                                                         │
│ Okay, so the user asked about why they should care about the Streamable HTTP Transport as an application develo │
│                                                                                                                 │
│ First, I need to recall the transport protocols from the spec. The HTTP transport uses POST and SSE for bidirec │
│                                                                                                                 │
│ The user is an application developer, so they might be interested in deployment scenarios, scalability, and eas │
│                                                                                                                 │
│ I should highlight how HTTP/SSE allows running servers as remote services. This is good for distributed systems │
│                                                                                                                 │
│ Cross-platform support is important too. HTTP is universal, so it works across different environments, which is │
│                                                                                                                 │
│ Long-lived connections are a plus for real-time interactions. The Server-Sent Events part lets servers push upd │
│                                                                                                                 │
│ Also, session management with headers like Mcp-Session-Id helps with maintaining state across requests, which i │
│                                                                                                                 │
│ Security aspects like using HTTPS and bearer tokens might be relevant. The spec mentions OAuth 2.1 for HTTP, wh │
│                                                                                                                 │
│ Support for multiple concurrent streams and resumable connections could enable complex interactions without mul │
│                                                                                                                 │
│ I should structure the answer around these points: remote deployment, cross-platform compatibility, real-time u │
│ </think>                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── 🤖 Assistant (Final Answer) ──────────────────────────────────────────╮
│ As an application developer, you should care about the **Streamable HTTP Transport** (HTTP/SSE) in the          │
│ 2025-03-26 MCP specification for the following **critical reasons**:                                            │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ### 1. **Remote Server Deployment = Cloud-Native Flexibility**                                                  │
│ - **Why It Matters**: The HTTP/SSE transport allows MCP servers to run as **independent remote services**       │
│ rather than local subprocesses. This decouples clients (like IDEs or web apps) from server logic, enabling:     │
│   - **Scalable microservices**: Deploy MCP servers as REST APIs for centralized tool execution (e.g., a         │
│ multi-tenant code-generation service).                                                                          │
│   - **Hybrid architectures**: Use MCP clients in mobile/desktop apps while servers run serverlessly (e.g., AWS  │
│ Lambda or Cloudflare Workers).                                                                                  │
│   - Example: A VS Code extension uses MCP over HTTP to connect to a remote Python LLM inference server,         │
│ avoiding local dependencies.                                                                                    │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ### 2. **Cross-Platform Compatibility**                                                                         │
│ - **Why It Matters**: HTTP/SSE is universally supported across **web, mobile, and cloud environments**:         │
│   - **Web apps**: Use the MCP client directly in browsers via JavaScript SDKs.                                  │
│   - **Mobile apps**: Integrate the HTTP transport in Android/iOS apps using standard networking libraries       │
│ (e.g., Retrofit, Alamofire).                                                                                    │
│   - **No language/runtime lock-in**: Unlike STDIO (which often requires Node.js/Python), HTTP/SSE works with    │
│ any backend stack (Java, Go, Rust, etc.).                                                                       │
│                                                                                                                 │
│ ---                                                                                                             │
│                                                                                                                 │
│ ### 3. **Bidirectional Streaming with Server-Sent Events (SSE)**                                                │
│ - **Why It Matters**: The HTTP transport supports **server-initiated streaming** via SSE, enabling real-time,   │
│ LLM-assisted workflows:                                                                                         │
│   ```mermaid                                                                                                    │
│   sequenceDiagram                                                                                               │
│     participant Client                                                                                          │
│     participant Server                                 